# 03 - Hexgrid creation

This notebook __creates and uploads the hexgrid on different resolutions__ for the area of interest created.

## __Import libraries__

In [1]:
from pathlib import Path

current_path = Path().resolve()

for parent in current_path.parents:
    if parent.name == "accesibilidad-urbana":
        project_root = parent
        break

print(project_root)

/home/observatorio/Documents/repos/accesibilidad-urbana


In [2]:
import os
import sys

import pandas as pd
import geopandas as gpd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns

from pandas.api.types import CategoricalDtype

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join(project_root))
if module_path not in sys.path:
    print(f"Path found.")
    sys.path.append(module_path)
    import aup
else:
    print(f"Path already loaded.")
    import aup

repo_path = str(module_path)+"/"
print(f"Repo path: {repo_path}")

Path already loaded.
Repo path: /home/observatorio/Documents/repos/accesibilidad-urbana/


## __Notebook config__

In [16]:
# ----- ----- ----- Process configuration
aoi_name = 'taoyuan_drawn_aoi'
res_list = [8,9,10,11]

# ----- ----- ----- Saving configuration
save_schema = 'projects_research'
hexgrid_save_table = 'taoyuan_hexgrid'
# save output to database?
save = True
if_exists='append'

## __Load area of interest__

In [14]:
# Load area of interest
aoi_schema = 'projects_research'
aoi_table = 'taoyuan_aoi'

query = f"SELECT * FROM {aoi_schema}.{aoi_table} WHERE \"aoi_name\" LIKE \'{aoi_name}\'"
aoi_gdf = aup.gdf_from_query(query, geometry_col='geometry')
aoi_gdf = aoi_gdf.set_crs("EPSG:4326")

# Show
print(aoi_gdf.crs)
print(aoi_gdf.shape)
aoi_gdf.head(1)

EPSG:4326
(1, 2)


,geometry,aoi_name
0,"MULTIPOLYGON (((121.26668 25.12071, 121.26901 ...",taoyuan_drawn_aoi


## __Create and upload (if save=True) hexgrid__

In [15]:
for res in res_list:

    print(f"Loading hexgrid res {res} for aoi {aoi_name}.")

    # Create hexgrid
    hexgrid_res = aup.create_hexgrid(aoi_gdf, hex_res=res)

    # Set res and aoi_name column
    hexgrid_res['res'] = res
    hexgrid_res.rename(columns={f'hex_id_{res}':'hex_id'},inplace=True)
    hexgrid_res['aoi_name'] = aoi_name

    # Reorder columns
    hexgrid_res = hexgrid_res[['hex_id','res','aoi_name','geometry']]

    # Upload
    if save: 
        print(f"Uploading hexgrid res {res} for aoi {aoi_name}.")
        aup.gdf_to_db_slow(hexgrid_res, hexgrid_save_table, save_schema, if_exists=if_exists)

# Show
print(hexgrid_res.crs)
print(hexgrid_res.shape)
hexgrid_res.head(1)

Loading hexgrid res 8 for aoi taoyuan_drawn_aoi.
Uploading hexgrid res 8 for aoi taoyuan_drawn_aoi.
Loading hexgrid res 9 for aoi taoyuan_drawn_aoi.
Uploading hexgrid res 9 for aoi taoyuan_drawn_aoi.
Loading hexgrid res 10 for aoi taoyuan_drawn_aoi.
Uploading hexgrid res 10 for aoi taoyuan_drawn_aoi.
Loading hexgrid res 11 for aoi taoyuan_drawn_aoi.
Uploading hexgrid res 11 for aoi taoyuan_drawn_aoi.
EPSG:4326
(1037498, 4)


,hex_id,res,aoi_name,geometry
0,8b4ba56961b3fff,11,taoyuan_drawn_aoi,"POLYGON ((121.50602 25.1279, 121.50607 25.1276..."
